In [1]:
import pydeck as pdk
import pandas as pd

with open('../../../mapbox_key.txt', 'r') as f:
    mapbox_key = f.readline()
    print('Mapbox key =', mapbox_key)

Mapbox key = pk.eyJ1IjoiZWp3aWxsZW1zZSIsImEiOiJjazhxMG1wYTkwMGMwM3Jwa29qcTduYjBzIn0.Ak0jgr96rC-W5WB1jlmkyw


# Plotting lights at night

NASA has collected global light emission data for over 30 years. The data set is a deeply fascinating one and has been used for news stories on the Syrian Civil War [[1]](https://time.com/3741451/syria-lights-civil-war-satellite/), North Korea [[2]](https://www.cbsnews.com/pictures/north-korea-hermit-country-space-photos/4/), and economic growth [[3]](https://qz.com/959563/nasas-black-marble-map-shows-the-light-of-human-population-centers-at-night-throughout-2016/).

In this notebook, we'll use a deck.gl [HeatmapLayer](https://deck.gl/#/documentation/deckgl-api-reference/layers/heatmap-layer) to visualize some of the changes at different points in time.

## Getting the data

The data for Chengdu, China, is cleaned and available below. Please note this data is meant for demonstration only.

In [2]:
LIGHTS_URL = 'https://raw.githubusercontent.com/ajduberstein/lights_at_night/master/chengdu_lights_at_night.csv'
df = pd.read_csv(LIGHTS_URL)
df.head()

,year,lng,lat,brightness
0,1993,104.575,31.808,4
1,1993,104.583,31.808,4
2,1993,104.592,31.808,4
3,1993,104.600,31.808,4
4,1993,104.675,31.808,4


### Setting the colors
pydeck does need to know the color for this data in advance of plotting it

In [3]:
df['color'] = df['brightness'].apply(lambda val: [255, val * 4,  255, 255])
df.sample(10)

,year,lng,lat,brightness,color
211752,2007,103.742,29.942,4,"[255, 16, 255, 255]"
221083,2011,104.392,31.500,5,"[255, 20, 255, 255]"
302623,1999,104.158,31.233,6,"[255, 24, 255, 255]"
97976,2001,103.975,30.833,9,"[255, 36, 255, 255]"
189866,2007,105.117,31.083,5,"[255, 20, 255, 255]"
106371,2001,104.517,30.350,7,"[255, 28, 255, 255]"
245706,2011,103.950,30.258,5,"[255, 20, 255, 255]"
43725,1995,104.300,30.883,8,"[255, 32, 255, 255]"
105197,2001,103.875,30.425,8,"[255, 32, 255, 255]"
117006,2003,104.742,31.617,3,"[255, 12, 255, 255]"


## Plotting and interacting

We can plot this data set of light brightness by year, configuring a slider to filter the data as below:

In [4]:
plottable = df[df['year'] == 1993].to_dict(orient='records')

view_state = pdk.ViewState(
    latitude=31.0,
    longitude=104.5,
    zoom=8,
    max_zoom=8,
    min_zoom=8)
scatterplot = pdk.Layer(
    'HeatmapLayer',
    data=plottable,
    get_position='[lng, lat]',
    get_weight='brightness',
    opacity=0.5,
    pickable=False,
    get_radius=800)
r = pdk.Deck(
    layers=[scatterplot],
    initial_view_state=view_state,
    views=[pdk.View(type='MapView', controller=None)],
    mapbox_key=mapbox_key)
r.show()

DeckGLWidget(json_input='{"initialViewState": {"latitude": 31.0, "longitude": 104.5, "maxZoom": 8, "minZoom": …

In [8]:
import ipywidgets as widgets
from IPython.display import display
slider = widgets.IntSlider(1992, min=1993, max=2013, step=2)
def on_change(v):
    results = df[df['year'] == slider.value].to_dict(orient='records')
    scatterplot.data = results
    r.update()
    
slider.observe(on_change, names='value')
display(slider)

IntSlider(value=1993, max=2013, min=1993, step=2)